In [7]:
# Installations
#!pip install gzip
#!pip install geopy
#!pip install boto

^C
  Using cached geopy-2.0.0-py3-none-any.whl (111 kB)
  Using cached geographiclib-1.50-py3-none-any.whl (38 kB)


You should consider upgrading via the 'c:\users\hp\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\hp\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [1]:
#jsonlines code
# Based on:
import jsonlines
import pprint
import json
import numpy as np
import pandas as pd
import gzip
import os
#Visualization of Final Results
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string

#Allowing for full text visibility
pd.set_option('display.max_colwidth', -1)

In [ ]:
#Get all relevant folders across all subdirectories
dir_name = 'D:\Dropbox\Programming\COVID_Religion_Project\COVID-19-TweetIDs\gz_unzip_test'
extension = ".gz"

#os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        print(item)
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

In [ ]:
#for later upgrade to deal with gzip files, rather than just jsonl
file = 'D:\Dropbox\Programming\COVID_Religion_Project\COVID-19-TweetIDs\gz_unzip_test\coronavirus-tweet-id-2020-01-21-22.jsonl.gz'
f = gzip.open(file, 'rb')
file_content = f.readlines()
f.close()

#Create list of dictionaries
final_json = [json.loads(line.decode('utf8')) for line in file_content]


In [7]:
# Select the columns that are of interest from the JSON Schema
keep_cols = ['created_at','favorite_count', 'full_text', 'id_str', 'lang', 'retweet_count', 'user']
user_keep_cols = ['created_at', 'followers_count', 'id_str', 'description', 'location', 'verified']

In [25]:
def jsonl_to_jsonlist(link):
    """
    Takes in a compressed json file, returns a list of json dictionary objects
    Deletes the entitites and retweeted_status keys in the dictionary whenever present
    Also removes the 'entities' subdictionary to limit the depth of nesting to 1
    """
    json_list =[]
    with jsonlines.open(link) as f:        
        for line in f.iter():
            modified_dict = {}
            for k, v in line.items():
                if k in keep_cols:
                    modified_dict[k] = v                    
                json_list.append(modified_dict)
    return json_list

In [26]:
jul_json = jsonl_to_jsonlist(jul_link)

In [ ]:
 try:
                del line['entities']
            except:
                pass            
            
            try:
                del line['retweeted_status']
            except:
                #since some tweets were never retweeted
                pass
            #we don't want unnecessary multi-level nesting
            for key in line['user'].keys():
                if isinstance(line['user'][key], list):
                    line['user'][key] = line['user'][key][0]
            

In [3]:
#Taking sample files from the beginning of Corona cases (March) and the present day
#These jsonl files were produced from 
mar_link = 'COVID-19-TweetIDs\gz_unzip_test\coronavirus-tweet-id-2020-03-08-17.jsonl'
jul_link = 'COVID-19-TweetIDs\gz_unzip_test\coronavirus-tweet-id-2020-07-01-05.jsonl'
may_link = 'COVID-19-TweetIDs\gz_unzip_test\coronavirus-tweet-id-2020-05-01-10.jsonl'

In [5]:
all_links = [mar_link, may_link, jul_link]

## Understanding the Shape of the Data

In [6]:
mar_json, may_json, jul_json = (jsonl_to_jsonlist(link) for link in all_links)

In [28]:
test = jul_json[90]
pprint.pprint(test)

{'created_at': 'Wed Jul 01 04:59:56 +0000 2020',
 'favorite_count': 1,
 'full_text': 'Prehospitalización de pacientes COVID en la atención primaria: '
              'La otra cara del colapso en el Hospital San José '
              'https://t.co/VTbNGS2NtD',
 'id_str': '1278191585474338816',
 'lang': 'es',
 'retweet_count': 3}


In [40]:
mar_json = jsonl_to_jsonlist(mar_link)

In [24]:
test = jul_json[1]
#The choice of 39 is random, any number within the length of the JSON list will suffice
pprint.pprint(test)

IndexError: list index out of range

## Data Normalization
We need to convert the nested json structure to a flat one. For this to work, we first find what components of the schema will be most useful for subsequent analysis.  

In [47]:
cols = list(test.keys())
cols.remove('user')
print(len(cols))
print(cols)

22
['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang']


In [48]:
#normalizing json
# https://stackoverflow.com/questions/40588852/pandas-read-nested-json
from pandas.io.json import json_normalize 
df_normal = json_normalize(mar_json,
                           record_path=['user'],
                           meta=cols,
                           #record_prefix='user_',
                           #meta_prefix='tweet_',
                           errors='ignore')

MemoryError: 

In [45]:
df_normal

,0
0,id
1,id_str
2,name
3,screen_name
4,location
5,description
6,url
7,entities
8,protected
9,followers_count


In [ ]:
len(df_normal.columns)

# Analyzing Tweet Content

In [25]:
import pandas as pd


# Some Functions from Last Time to get us started:
def get_wordnet_pos(word):

    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": nltk.corpus.wordnet.ADJ,
                "N": nltk.corpus.wordnet.NOUN,
                "V": nltk.corpus.wordnet.VERB,
                "R": nltk.corpus.wordnet.ADV}

    return tag_dict.get(tag, nltk.corpus.wordnet.NOUN)

def get_lemmas(text):
    stop = nltk.corpus.stopwords.words('english') + list(string.punctuation)
    tokens = [i for i in nltk.word_tokenize(text.lower()) if i not in stop]
    lemmas = [nltk.stem.WordNetLemmatizer().lemmatize(t, get_wordnet_pos(t)) for t in tokens]
    return lemmas

def get_tokens(text):
    # drop punctuation, but keep stopwords for initial word counting
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = [i for i in nltk.word_tokenize(text.lower())]
    return tokens

In [12]:
def json_to_tweet_text(json_lst):
    df = pd.DataFrame(json_lst)
    df_eng = df[df['lang']=='en']
    #tweet_text = df_eng['full_text'].apply(get_tokens)
    return df_eng['full_text']


In [30]:
# Checking for the religious statements
may_df = df = pd.DataFrame(may_json)
may_df['relig'] = may_df['full_text'].apply(lambda x: relig_perc(x, relative_words))

In [36]:
may_df[may_df['relig']==True]['full_text']

1219    RT @misskaul: A short thread on death during a lockdown. Anyone affected, Bollywood stars, or your father in law, beloved doctor for much o…                                                                                                      
5163    @sukumaranlens @JMehta65 U rush to your father's bedside at night ..stay by his side all night .. he dies in the morning ... Will u rush home to change your clothes... What an idiotic hypocrite u r. Guys like u make me puke. Fckng get a Life.
Name: full_text, dtype: object

In [64]:
mar_text = json_to_tweet_text(mar_json)

In [7]:
jul_text = json_to_tweet_text(jul_json)

In [13]:
may_text = json_to_tweet_text(may_json)

In [14]:
may_text

0        RT @net_drifter: Oh so today, MewGulf had gone...
1        #うひーメモ\n投稿時間:2020-05-01 19:16:50\nCoronavirus ...
2        RT @RobynUrback: To thinking, breathing, moder...
3            RT @faizwahab: Majikan won't care. Believe me
5        RT @DrGJackBrown: • We do not live for Trump's...
7        RT @PodSaveAmerica: Trump dismisses coronaviru...
11         @GovMurphy We should maintain social distancing
12       @StudentU_UK Looking for Full Refund from Airl...
13       RT @ak_annanavarro: there’s doctors literally ...
14       RT @CGTNOfficial: The first results from human...
15       @BristOliver This is an incredible stat, for e...
18       RT @RealErinCruz: No, @GavinNewsom ... you are...
19       RT @Rpsingh5153P: My sincere and humble greeti...
20       My dear best friend died from Covid19. This is...
21       RT @dhanyarajendran: In the last few days, the...
24       RT @CBSNews: The family of this 4-year-old say...
26       RT @EK_Drake: Refreshing to see such an honest.

In [15]:
relative_vocab = pd.read_csv('relatedQueries.csv')
relative_words = list(relative_vocab['word'])

In [9]:
print(relative_words)

['the prayer', 'prayer times', 'prayer time', 'catholic prayer', 'healing prayer', 'serenity prayer', 'serenity', 'what is prayer', 'lords prayer', 'morning prayer', 'prayer for healing', 'jesus prayer', 'daily prayer', 'our father prayer', 'our father', 'prayer quotes', 'muslim prayer', 'lord prayer', 'prayer plant', 'the lords prayer', 'islamic prayer', 'hail mary', 'hail mary prayer', 'house of prayer', 'night prayer']


In [16]:
rising_vocab = pd.read_csv('risingQueries.csv')
rising_words = list(rising_vocab['word'])

In [11]:
print(rising_words)

['coronavirus prayer', 'prayer for coronavirus', 'national day of prayer 2020', 'ramadan 2020', 'prayer of spiritual communion', 'spiritual communion prayer', 'prayer for spiritual communion', 'lds sacrament prayer', 'the hunters prayer', 'let this be our prayer', 'sacrament prayer', 'national prayer breakfast', 'communion prayer', 'divine mercy chaplet prayer', 'divine mercy prayer', 'morning prayer for him', 'prayer for the world', 'prayer plant care', 'national day of prayer', 'prayer before bed', 'daily prayer us', 'assembly of prayer', 'prayer plant', 'hail holy queen prayer', 'isha prayer times']


In [17]:
relative_words.extend(rising_words)

In [18]:
print(relative_words)

['the prayer', 'prayer times', 'prayer time', 'catholic prayer', 'healing prayer', 'serenity prayer', 'serenity', 'what is prayer', 'lords prayer', 'morning prayer', 'prayer for healing', 'jesus prayer', 'daily prayer', 'our father prayer', 'our father', 'prayer quotes', 'muslim prayer', 'lord prayer', 'prayer plant', 'the lords prayer', 'islamic prayer', 'hail mary', 'hail mary prayer', 'house of prayer', 'night prayer', 'coronavirus prayer', 'prayer for coronavirus', 'national day of prayer 2020', 'ramadan 2020', 'prayer of spiritual communion', 'spiritual communion prayer', 'prayer for spiritual communion', 'lds sacrament prayer', 'the hunters prayer', 'let this be our prayer', 'sacrament prayer', 'national prayer breakfast', 'communion prayer', 'divine mercy chaplet prayer', 'divine mercy prayer', 'morning prayer for him', 'prayer for the world', 'prayer plant care', 'national day of prayer', 'prayer before bed', 'daily prayer us', 'assembly of prayer', 'prayer plant', 'hail holy q

In [19]:
#Measure religiosity
def relig_perc(text, vocab):
    return any([word in text for word in vocab])


In [15]:
mar_relig = mar_text.apply(lambda x: relig_perc(x, relative_words))

NameError: name 'mar_text' is not defined

In [20]:
may_relig = may_text.apply(lambda x: relig_perc(x, relative_words))

In [27]:
print(may_relig)

0        False
1        False
2        False
3        False
5        False
7        False
11       False
12       False
13       False
14       False
15       False
18       False
19       False
20       False
21       False
24       False
26       False
27       False
29       False
30       False
32       False
33       False
34       False
36       False
37       False
38       False
41       False
42       False
43       False
44       False
         ...  
25750    False
25755    False
25756    False
25757    False
25759    False
25764    False
25767    False
25770    False
25772    False
25773    False
25774    False
25776    False
25779    False
25780    False
25781    False
25782    False
25783    False
25785    False
25786    False
25787    False
25788    False
25790    False
25791    False
25792    False
25793    False
25794    False
25796    False
25797    False
25798    False
25800    False
Name: full_text, Length: 14602, dtype: bool


In [21]:
print(may_relig.mean())

0.00013696753869332968


In [16]:
jul_relig = jul_text.apply(lambda x: relig_perc(x, relative_words))

In [19]:
print(jul_relig.mean())

9.942169712836999e-05


In [24]:
def jsonl_to_mean_relig(link):
    """
    
    Arguments: The location of the original json file
    Returns: The percentage of tweets which have at least one of the religion words
    """
    relig_series = json_to_tweet_text(jsonl_to_jsonlist(link)).apply(lambda x: relig_perc(x, relative_words)) 
    return relig_series.mean()
    

In [25]:
print(jsonl_to_mean_relig(may_link))

0.00013696753869332968


In [26]:
print(jsonl_to_mean_relig(jul_link))

9.942169712836999e-05


# Extracting the Geocodes

In [ ]:
from geopy.geocoders import OpenMapQuest
from geopy.extra.rate_limiter import RateLimiter
geolocator = OpenMapQuest(api_key='FGtjwBYCTEadGZpAIEkM2ySw5518bS7b') 
geocoder = RateLimiter(geolocator.geocode, min_delay_seconds=1) 
fwp_df['location'] = fwp_df['userlocation'].apply(geocoder)